In [7]:
import time
from dotenv import load_dotenv
import os

load_dotenv()  # .env 파일 로드
hf_token = os.getenv('HF_TOKEN')


In [3]:
import pandas
import torch

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM


c:\repo\finetune_sogm123\sogm123_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# BitsAndBytesConfig 설정
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 8비트 대신 4비트 사용
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    #device_map="cuda",
    trust_remote_code=True
)

from peft import prepare_model_for_kbit_training
model = prepare_model_for_kbit_training(model)

model.gradient_checkpointing_enable()


`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.61s/it]


In [10]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((409

In [11]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations

In [13]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable : {100 * trainable_params / all_param}%"
    )
    return trainable_params

In [14]:
ori_p = print_trainable_parameters(model)

trainable params: 0 || all params: 4,540,600,320 || trainable : 0.0%


In [15]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, #attention heads
    lora_alpha=32, #alpha scaling
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CAUSAL LANGUAGE MODELS (like Bloom, LLaMA) or SEQ TO SEQ (like FLAN, T5)
)

model = get_peft_model(model, config)

In [16]:
peft_p = print_trainable_parameters(model)

trainable params: 6,815,744 || all params: 4,547,416,064 || trainable : 0.14988168894325302%


In [18]:
!pip install datasets

  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/25.2 MB ? eta -:--:--
   ---- ----------------------------------- 2.6/25.2 MB 13.1 MB/s eta 0:00:02
   --------- ------------------------------ 6.3/25.2 MB 15.6 MB/s eta 0:00:02
   ---------------- ----------------------- 10.5/25.2 MB 17.4 MB/s eta 0:00:01
   ---------------- ----------------------- 10.5/25.2 MB 17.4 MB/s eta 0:00:01
   -------------------- ------------------- 13.1/25.2 MB 12.5 MB/s eta 0:00:01
   ------------------------------------ --- 23.3/25.2 MB 18.5 MB/s eta 0:00:01
   ---------------------------------------- 25.2/25.2 MB 18.8 MB/s eta 0:00:00
Using cached attrs-25.1.0-py3-none-any.whl (63 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.2.0
    Uninstalling fsspec-2025.2.0:
      Successfully uninstalled fsspec-2025.2.0



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
import transformers
from datasets import load_dataset
dataset = load_dataset("kimjaewon/baemin_sft_data")

c:\repo\finetune_sogm123\sogm123_venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\datasets--kimjaewon--baemin_sft_data. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating train split: 100%|██████████| 1452/1452 [00:00<00:00, 156792.37 examples/s]


In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'positive_document_list', 'negative_document_list', 'answer'],
        num_rows: 1452
    })
})

In [20]:
peft_dataset = dataset.remove_columns(['positive_document_list', 'negative_document_list'])
peft_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1452
    })
})

In [21]:
peft_dataset['train']['question'][:8]

['정산이 이뤄지는 시점은 언제인가요?',
 '주문 차단을 하고 싶다면 어떻게 해야 하나요?',
 '탈퇴 회원의 댓글 작성자 본인 여부를 확인할 수 있는 방법은 없나요?',
 '네이버 플레이스에 가게 정보를 제공하면 연동 여부 반영까지 얼마나 걸리나요?',
 '어떤 음식은 배달의민족을 통해 판매할 수 없나요?',
 '어떤 경우에 고객센터로 문의해야 하나요?',
 '울트라콜과 오픈리스트 상품의 배달팁은 어떻게 구분되나요?',
 '배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 가능한가요?']

In [22]:
def merge_cols(example):
    example["prediction"] = example["question"] + " ===> " + example["answer"]
    return example

peft_dataset['train'] = peft_dataset['train'].map(merge_cols) # <-- 모든 문장에 대해 처리해 줍니다.
peft_dataset['train']["prediction"][5:7]

Map: 100%|██████████| 1452/1452 [00:00<00:00, 31617.82 examples/s]


['어떤 경우에 고객센터로 문의해야 하나요? ===> 고객의 개인정보와 관련된 문의와 함께 배달의 불만사항, 결제문제 등 배달 서비스와 관련된 문제가 발생한 경우, 그리고 배달 관련 추가 연락이 필요한 경우에는 고객센터로 문의해주시면 최대한 도움을 드리겠습니다.',
 '울트라콜과 오픈리스트 상품의 배달팁은 어떻게 구분되나요? ===> 울트라콜과 오픈리스트 상품의 배달팁은 용도에 따라 기본 배달팁과 할증 배달팁으로 나누어집니다. 기본 배달팁은 주문금액에 따른 최대 3개의 배달팁 설정이 가능하며, 할증 배달팁은 지역, 시간대, 공휴일 등을 위해 별도로 설정이 가능합니다.']

In [23]:
peft_dataset['train'][7]

{'question': '배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 가능한가요?',
 'answer': '아니요, 배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 불가능합니다. 배달지역 조회만 가능하며, 배달지역 수정이 필요한 경우 고객센터를 통해 문의하셔야 합니다.',
 'prediction': '배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 가능한가요? ===> 아니요, 배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 불가능합니다. 배달지역 조회만 가능하며, 배달지역 수정이 필요한 경우 고객센터를 통해 문의하셔야 합니다.'}

In [24]:
peft_dataset = peft_dataset.map(
                        lambda x: tokenizer(x['prediction']),
                        batched=True
                     )

Map: 100%|██████████| 1452/1452 [00:00<00:00, 17370.00 examples/s]


In [25]:
peft_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 1452
    })
})

In [32]:
train_args = transformers.TrainingArguments(
    output_dir='outputs',
    
    # 학습 관련 설정
    per_device_train_batch_size=1,      # 배치 사이즈 1로 설정
    gradient_accumulation_steps=8,       # 8번 누적 (효과적 배치 사이즈 = 8)
    learning_rate=2e-4,
    
    # 최적화 관련 설정
    fp16=True,                          # fp16 사용
    optim="adamw_torch_fused",          # 메모리 효율적인 옵티마이저
    
    # 스케줄링
    warmup_steps=100,
    max_steps=20,
    
    # 로깅
    logging_steps=20,
)

tokenizer.pad_token = tokenizer.eos_token 
model=model.to("cuda")

trainer = transformers.Trainer(
    model=model,
    train_dataset=peft_dataset['train'],
    args=train_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [28]:
!nvidia-smi

Sun Feb 16 03:09:12 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.18                 Driver Version: 531.18       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080       WDDM | 00000000:01:00.0  On |                  N/A |
| 31%   48C    P5               46W / 370W|   9718MiB / 10240MiB |     13%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [33]:
trainer.train()

Step,Training Loss
20,2.839500


TrainOutput(global_step=20, training_loss=2.8395259857177733, metrics={'train_runtime': 118.7224, 'train_samples_per_second': 1.348, 'train_steps_per_second': 0.168, 'total_flos': 565408701112320.0, 'train_loss': 2.8395259857177733, 'epoch': 0.11019283746556474})

In [36]:
# 새로운 amp 문법 사용 및 디바이스 이동
user_message = tokenizer("배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 가능한가요? ===> 한국어로 대답하세요", return_tensors='pt')
user_message = {k: v.to('cuda') for k, v in user_message.items()}  # 모든 입력을 GPU로

with torch.amp.autocast('cuda'):  # 새로운 문법
    output_tokens = model.generate(
        **user_message,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=128,
        do_sample=True,          # 다양한 응답을 위해
        temperature=0.7,         # 창의성 조절
        top_p=0.9               # nucleus sampling
    )

print('peft_ai : ', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

peft_ai :  배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 가능한가요? ===> 한국어로 대답하세요.

배민라이더스는 배민셀프서비스를 통한 배달지역 수정이 불가능합니다. 배민라이더스는 배달지역 수정이 불가능한 기본적인 서비스입니다. 배민셀프서비스는 배민라이더스 외의 서비스입니다. 배민셀프서비스를 사용하여 배달지역을 수정하려면 배민셀프서비스를 사용하여야 합니다.

Translation:

Can I modify the delivery area through Minim Self-Service as a Minim Rider? ===> Answer in Korean.

As a Minim Rider, you cannot modify the delivery area through Minim Self-Service. The


In [30]:
model_path = 'llama3_peft'  # it will be directory
model.save_pretrained(model_path)

/home/freenak/miniconda3/envs/llm_env/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
lora_config = LoraConfig.from_pretrained(model_path)
model = get_peft_model(model, lora_config)